# IBM Data Science Final Capstone Project

 ## Which London boroughs provide the best amenities such as coffee shops, resturants and parks. But also have low or lower crime rates than other London boroughs



## Introduction

London is a large and vibrant city, with a population of around 9 million people. It is popular not only as the Capital of the United Kingdom and the work oppurtunities it provides,  but also becuase it provides great services and amentities. London is a city with many coffee shops, great resturants and place to visit, it is also a green city with parks and open spaces which make it attractive to people looking to live and work there. 

Of course the downside of any large city is Crime, in 2018/19 there were a total of 912,000 crimes across London.


## Problem Description

London is comprised of 32 Boroughs and the City of London (not classed as a Borough), if someone was moving to London which Borough should they choose to search for a place to live, obviously they would like to have the advantages which London brings such as great places to eat and drink, have a coffee and be able to get out and enjoy parks for recreation and excercise, but also to have a low crime rate.

To answer this question we will utilize the FourSquare API to provide data on the key catergories of venues, details of the London boroughs from wikipedia, and London Crime Data from Uk Government

 ## Data Collection

We will use the following three datasets to analyse the London boroughs

London Crime data is available to dowload at https://data.london.gov.uk/dataset/recorded_crime_summary, this provides a breakdown of crimes at different geographical levels within london
    
Lets load the libaries and download the crime dataset

In [3]:
import pandas as pd
import io
import requests

url="https://data.london.gov.uk/download/recorded_crime_summary/d2e9ccfc-a054-41e3-89fb-53c2bc3ed87a/MPS%20Borough%20Level%20Crime%20%28most%20recent%2024%20months%29.csv"
s=requests.get(url).content
df_crime=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [5]:
# lets take a quick look at the data provided
print('How many rows and columns do we have',df_crime.shape)
df_crime.head()

How many rows and columns do we have (1569, 27)


,MajorText,MinorText,LookUp_BoroughName,201807,201808,201809,201810,201811,201812,201901,...,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006
0,Arson and Criminal Damage,Arson,Barking and Dagenham,6,5,3,8,5,1,5,...,6,9,8,6,4,5,6,2,2,4
1,Arson and Criminal Damage,Criminal Damage,Barking and Dagenham,127,101,107,132,105,88,97,...,109,109,97,121,97,103,107,80,86,121
2,Burglary,Burglary - Business and Community,Barking and Dagenham,30,18,33,32,39,33,45,...,37,30,30,25,31,17,28,29,16,16
3,Burglary,Burglary - Residential,Barking and Dagenham,94,84,99,94,106,164,114,...,80,97,114,130,116,123,97,57,41,63
4,Drug Offences,Drug Trafficking,Barking and Dagenham,8,7,10,7,7,4,5,...,7,8,13,3,14,5,6,12,12,7


The crime dataset provides a breakdown of types of crime per borough, for each month covering the last 24 months. We will use this data to catergorize the types and numbers of crime for each borough to identify which boroghs have lower rates of crime

The next dataset we will use is the wikipedia resource which provieds a list of all the London boroughs, which is available at https://en.wikipedia.org/wiki/List_of_London_boroughs, handily this dataset includes the latitude and longitude of each borough, along with population estimates and the area of the borough. We will use the lat/long as input to the Foursquare API to find venues in the local area, the population will be used to calcualte the crimes per 1000 residents

Lets download the data from Wikipedia

In [6]:
df_boroughs = pd.read_html('https://en.wikipedia.org/wiki/List_of_London_boroughs')[0]

In [7]:
df_boroughs.head()

# luckily we also have the lat/lng in the dataset which will be great for foursquare!

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E,25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


We also want to include the City of London to make sure we have all of London included, so lets load that in too.

In [8]:
df_col = pd.read_html('https://en.wikipedia.org/wiki/List_of_London_boroughs')[1]

In [9]:
df_col

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population(2011 est),Co-ordinates,Nr. inmap
0,City of London,([note 5],Sui generis;City;Ceremonial county,Corporation of London;Inner Temple;Middle Temple,?,Guildhall,1.12,7000,51°30′56″N 0°05′32″W﻿ / ﻿51.5155°N 0.0922°W,1


 Lets add that to our list of boroughs to make a full dataset

In [10]:
df_london = df_boroughs.append(df_col, ignore_index=True, sort=False)

In [11]:
# lets check the data
df_london

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map,Population(2011 est),Nr. inmap
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352.0,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E,25.0,NaN,NaN
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,369088.0,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31.0,NaN,NaN
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687.0,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23.0,NaN,NaN
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264.0,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12.0,NaN,NaN
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899.0,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20.0,NaN,NaN
5,Camden,NaN,NaN,Camden London Borough Council,Labour,"Camden Town Hall, Judd Street",8.40,229719.0,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W,11.0,NaN,NaN
6,Croydon,NaN,NaN,Croydon London Borough Council,Labour,"Bernard Weatherill House, Mint Walk",33.41,372752.0,51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W,19.0,NaN,NaN
7,Ealing,NaN,NaN,Ealing London Borough Council,Labour,"Perceval House, 14-16 Uxbridge Road",21.44,342494.0,51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W,13.0,NaN,NaN
8,Enfield,NaN,NaN,Enfield London Borough Council,Labour,"Civic Centre, Silver Street",31.74,320524.0,51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W,30.0,NaN,NaN
9,Greenwich [note 2],[note 3],Royal,Greenwich London Borough Council,Labour,"Woolwich Town Hall, Wellington Street",18.28,264008.0,51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E,22.0,NaN,NaN


Looking at the Wikipedia data we will have to do some clean up, for example some borough names contain notes, and the lat long is in the format degrees, minutes north West etc so we will have to reformat these for the Foursquare API, we are also missing population for the city of london wikipedia https://en.wikipedia.org/wiki/City_of_London states that the population of the city of London was 7,375 in 2011 so we will add that into the dataset to make it complete

Finally we will use the Foursquare API to provide details of venues for each of the boroughs which we will analyze

We will use clustering to build a map of london to show similar boroughs based on numbers of venues and crime rates to indentify those boroughs which meet our criteria, specifically this criteria will be coffee shops, cafes, bars, resturants, Parks/Open Spaces and Crime Rates